In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 25)

In [2]:
stats = pd.read_csv("stats.csv")

def get_min_age(player):
    return stats[stats['Player'] == player]['Age'].min()

stats['min_age'] = stats['Player'].apply(get_min_age)

stats.dropna(subset=['TOTAL_MP'], inplace = True) # removes 609 seasons where the player didn't play at all
stats = stats[stats['TOTAL_MP'] >= 100]

def get_total_seasons(player):
    return stats[stats['Player'] == player].shape[0]

stats['TOTAL_seasons'] = stats['Player'].apply(get_total_seasons)

def index_seasons(player, year):
    season_list = list(stats[stats['Player'] == player]['Year'])
    season_list.sort()
    return season_list.index(year)

stats['season_index'] = [index_seasons(player, year) for player, year in zip(stats['Player'], stats['Year'])]

In [3]:
# convert minutes played to % of available minutes played
# helps to account for seasons shortened by injury or lockout

stats['min_proportion'] = stats['TOTAL_MP'] / (stats['G'] * 40)

In [4]:
stats['3P%'].fillna(value = 0.12, inplace = True)
def fix_low_3pa(attempts, rate):
    if attempts <= 5:
        return 0.15
    
    elif attempts <= 15:
        if rate >= 0.25:
            return 0.25
        else:
            return np.max([0.12, rate])
        
    elif attempts <= 25:
        if rate >= 0.35:
            return 0.35
        else:
            return np.max([0.1, rate])
        
    else:
        return rate

stats['3P%'] = [fix_low_3pa(attempts, rate) for attempts,
                                  rate in zip(stats['TOTAL_3PA'], stats['3P%'])]
def fix_low_fta(attempts, rate):
    if attempts <= 10:
        return 0.75
    
    elif attempts <= 25:
        if rate >= 0.8:
            return 0.8
        else:
            return np.max([0.7, rate])
        
    elif attempts <= 50:
        if rate >= 0.85:
            return 0.85
        else:
            return np.max([0.60, rate])
        
    else:
        return rate

stats['FT%'] = [fix_low_fta(attempts, rate) for attempts,
                                  rate in zip(stats['TOTAL_FTA'], stats['FT%'])]

def fix_low_2pa(attempts, rate):
    if attempts <= 25:
        if rate >= 0.6:
            return 0.6
        else:
            return np.max([0.3, rate])
    else:
        return rate

stats['2P%'] = [fix_low_fta(attempts, rate) for attempts,
                                  rate in zip(stats['TOTAL_2PA'], stats['2P%'])]


In [6]:
stats['two_point_attempts'] = stats['2PA_per_100']/75
stats['three_point_attempts'] = stats['3PA_per_100']/75
stats['free_throw_attempts'] = stats['FTA_per_100']/75
stats['defensive_rebounds'] = stats['DRB_per_100']/75
stats['offensive_rebounds'] = stats['ORB_per_100']/75
stats['assists'] = stats['AST_per_100']/75
stats['steals'] = stats['STL_per_100']/75
stats['blocks'] = stats['BLK_per_100']/75
stats['turnovers'] = stats['TOV_per_100']/75
stats['personal_fouls'] = stats['PF_per_100']/75

In [7]:
stats['3P%'] = stats['3P%'] * 1.8

In [8]:
def logit(p):
    if p > 0.95:
        p = 0.95
    if p < 0.05:
        p = 0.05
    return np.log(p/(1-p))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [9]:
stats['minutes_proportion'] = stats['min_proportion'].apply(logit)
stats['two_point_percentage'] = stats['2P%'].apply(logit)
stats['three_point_percentage'] = stats['3P%'].apply(logit)
stats['free_throw_percentage'] = stats['FT%'].apply(logit)

In [10]:
stats['Age2'] = stats['Age'] ** 2
stats['Age3'] = stats['Age'] ** 3

In [19]:
class MultiModelContainer:
    
    '''
    Object to function as a container for multiple linear models.
    Initialize with a dictionary where keys are targets and values are lists of features.
    '''
    
    def __init__(self, feature_dict, data):
        
        self.model_dict = {}
        self.feature_dict = feature_dict
        
        # initialize and fit models
        for target in feature_dict.keys():
            features = feature_dict[target]
            self.model_dict[target] = RandomForestRegressor(n_estimators = 75, max_depth = 6).fit(data[features].values, data[target].values)     

    def predict(self, targets, data, output_df = False, input_df = True, feature_map = None):
        
        switch = False
        
        for target in targets:
            # select features for target
            features = self.feature_dict[target]
            
            if input_df:
                # select model and input data for prediction
                preds = self.model_dict[target].predict(data[features])
            else:
                # if not using a df use feature map to select data from an array
                X = np.hstack(tuple([data[:,feature_map[feature]] for feature in features]))
                preds = self.model_dict[target].predict(X.reshape(-1,len(features)))
            
            if not switch:
                predictions = preds.reshape(-1,1)
                switch = True
                
            else:
                predictions = np.hstack((predictions, preds.reshape(-1,1)))
        
        if not output_df:
            return predictions
        
        else:
            return pd.DataFrame(predictions, columns = targets, index = data.index)

In [13]:
import pickle

with open('Models/FinalModelRFper75.pkl', 'rb') as file:
    FinalModel = pickle.load(file)
    
with open('Models/MinutesModelRFper75.pkl', 'rb') as file:
    MinutesModel = pickle.load(file)

In [14]:
features = ['Age', 'two_point_percentage', 'two_point_attempts', 'three_point_percentage',
       'three_point_attempts', 'free_throw_percentage', 'free_throw_attempts',
       'defensive_rebounds', 'offensive_rebounds', 'assists', 'steals',
       'blocks', 'turnovers', 'personal_fouls', 'Age2', 'Age3']

# map p1 features to column index of current version
p1_features = ['Age', 'two_point_percentage_p1', 'two_point_attempts_p1', 'three_point_percentage_p1',
       'three_point_attempts_p1', 'free_throw_percentage_p1', 'free_throw_attempts_p1',
       'defensive_rebounds_p1', 'offensive_rebounds_p1', 'assists_p1', 'steals_p1',
       'blocks_p1', 'turnovers_p1', 'personal_fouls_p1', 'Age2', 'Age3']

feature_map = {}

for feat in p1_features:
    feature_map[feat] = p1_features.index(feat)
    
targets = ['two_point_percentage', 'two_point_attempts', 'three_point_percentage',
       'three_point_attempts', 'free_throw_percentage', 'free_throw_attempts',
       'defensive_rebounds', 'offensive_rebounds', 'assists', 'steals',
       'blocks', 'turnovers', 'personal_fouls']

In [15]:
mask = np.array(stats['Year'] == 2021)
new_players = stats[mask]
new_players

,Unnamed: 0,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,...,offensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,minutes_proportion,two_point_percentage,three_point_percentage,free_throw_percentage,Age2,Age3
630,630,Aaron Gordon,PF,25,DEN,50,50,27.7,4.6,10.0,0.463,1.2,...,0.036000,0.076000,0.016000,0.016000,0.045333,0.042667,0.809486,0.132192,0.417981,0.623438,625,15625
631,631,Austin Rivers,SG,28,DEN,36,7,23.5,2.9,6.8,0.424,1.5,...,0.006667,0.064000,0.024000,0.001333,0.028000,0.050667,0.350775,0.040005,0.682068,0.895384,784,21952
632,632,Bol Bol,PF,21,DEN,32,2,5.0,0.8,1.8,0.431,0.3,...,0.000000,0.029333,0.012000,0.041333,0.053333,0.057333,-1.945910,0.405465,0.532217,0.847298,441,9261
633,633,Facundo Campazzo,PG,29,DEN,65,19,21.9,1.8,4.8,0.381,1.2,...,0.010667,0.106667,0.036000,0.006667,0.033333,0.061333,0.192904,-0.224944,0.547692,1.982994,841,24389
635,635,JaMychal Green,PF,30,DEN,58,5,19.3,3.0,6.4,0.463,1.4,...,0.050667,0.030667,0.014667,0.013333,0.030667,0.085333,-0.068993,0.136210,0.935550,1.430633,900,27000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13897,13897,Patty Mills,PG,32,SAS,68,1,24.8,3.7,9.0,0.412,2.4,...,0.006667,0.062667,0.016000,0.001333,0.025333,0.030667,0.487364,0.000000,0.730888,2.313635,1024,32768
13898,13898,Quinndary Weatherspoon,SG,24,SAS,20,0,6.1,0.8,1.8,0.457,0.1,...,0.016000,0.042667,0.042667,0.010667,0.053333,0.096000,-1.724831,0.405465,-0.844442,1.386294,576,13824
13899,13899,Rudy Gay,PF,34,SAS,63,1,21.6,4.2,10.0,0.420,1.7,...,0.020000,0.041333,0.021333,0.018667,0.030667,0.054667,0.155870,-0.196631,0.780556,1.411485,1156,39304
13900,13900,Tre Jones,PG,21,SAS,37,1,7.3,1.0,2.1,0.474,0.1,...,0.018667,0.093333,0.018667,0.000000,0.030667,0.030667,-1.504490,-0.136210,-0.994623,1.386294,441,9261


In [16]:
print(list(new_players['Player']))

['Aaron Gordon', 'Austin Rivers', 'Bol Bol', 'Facundo Campazzo', 'JaMychal Green', 'JaVale McGee', 'Jamal Murray', 'Markus Howard', 'Michael Porter Jr.', 'Monte Morris', 'Nikola Jokić', 'PJ Dozier', 'Paul Millsap', 'Shaquille Harrison', 'Vlatko Čančar', 'Will Barton', 'Zeke Nnaji', 'Aaron Holiday', 'Caris LeVert', 'Domantas Sabonis', 'Doug McDermott', 'Edmond Sumner', 'Goga Bitadze', 'JaKarr Sampson', 'Jeremy Lamb', 'Justin Holiday', 'Kelan Martin', 'Malcolm Brogdon', 'Myles Turner', 'Oshae Brissett', 'T.J. McConnell', 'T.J. Warren', 'Aaron Nesmith', 'Carsen Edwards', 'Evan Fournier', 'Grant Williams', 'Jabari Parker', 'Jaylen Brown', 'Jayson Tatum', 'Kemba Walker', 'Luke Kornet', 'Marcus Smart', 'Payton Pritchard', 'Robert Williams', 'Romeo Langford', 'Semi Ojeleye', 'Tacko Fall', 'Tremont Waters', 'Tristan Thompson', 'Abdel Nader', 'Cameron Johnson', 'Cameron Payne', 'Chris Paul', 'Dario Šarić', 'Deandre Ayton', 'Devin Booker', "E'Twaun Moore", 'Frank Kaminsky', 'Jae Crowder', 'Jalen

In [17]:
new_players.shape

(468, 119)

In [20]:
def predict_career(player_id, n_years, start_year, stats_model, minutes_model):
    
    # get the players actual data and create seed for predictions
    mask = np.array(stats['Player'] == player_id) & np.array(stats['Year'] == start_year)
    seed = np.array(stats[mask][features]).reshape(1,-1)
    mseed = np.array(stats[mask]['minutes_proportion']).reshape(-1,1)
    
    # iterate through total seasons and generate prediction based on each past prediction
    for i in range(0, n_years):
        
        # get inputs for prediction, previous season stats and CURRENT age
        pred_inputs = seed[-1,1:-2].reshape(1,-1)
        current_age = np.array(seed[i,0] + 1).reshape(1,-1)
        pred_inputs = np.hstack((current_age, pred_inputs, current_age**2, current_age**3))
        preds = stats_model.predict(targets, pred_inputs, input_df = False, feature_map = feature_map)
        
        # append age features to prediction
        preds = np.hstack((current_age, preds, current_age**2, current_age**3))
        
        # append predictions to seed
        seed = np.vstack((seed, preds))
    
    # predict minutes in each season
    for i in range(0, n_years):
        pred_inputs = np.hstack((seed[i+1,:], mseed[-1,:])).reshape(1,-1)
        minutes = minutes_model.predict(pred_inputs)
        mseed = np.vstack((mseed, minutes.reshape(-1,1)))
        
    seed = np.hstack((mseed, seed))
    
    # Convert logits back to percentages
    seed[:,0] = sigmoid(seed[:,0])
    
    for i, feat in enumerate(features):
        if feat[-10:] == 'percentage':
            seed[:,i + 1] = sigmoid(seed[:,i + 1])
    
    # Remove age2 and age3 features
    seed = seed[:,:-2]
    
    # Output Formatting
    cols = ['minutes_proportion'] + features[:-2]
    seed = pd.DataFrame(np.round(seed, 3), columns = cols)
    seed['Player'] = player_id
    #seed['name'] = seasons[mask]['name'][0]
    seed['Year'] = [start_year + i for i in range(n_years + 1)]
    seed['index_col'] = [str(player_id) + str(year) for player_id, year in zip(seed['Player'], seed['Year'])]
    seed.set_index('index_col', inplace = True)
    
    # Adjust 3pt percentage by dividing by 1.8
    seed['three_point_percentage'] = seed['three_point_percentage'] / 1.8
    
    #order columns for output
    final_cols = ['Player', 'Year'] + cols
    
    return seed[final_cols]

In [21]:
career_preds = {}

for player_id in list(new_players['Player']):
    career_preds[player_id] = predict_career(player_id, 5, 2021, FinalModel, MinutesModel)

C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\ba

C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\ba

C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\ba

C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\ba

C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\ba

...

C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\ba

C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\ba

C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\ba

C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\ba

C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\Cillian\anaconda3\lib\site-packages\sklearn\ba

In [22]:
from copy import deepcopy

In [129]:
def per_game_averages(player_df):
    pg = deepcopy(player_df)
    
    pg['minutes_per_game'] = pg['minutes_proportion'] * 40
    pg.drop(columns = 'minutes_proportion', inplace = True)
    
    pg['two_pointers'] = pg['two_point_attempts'] * pg['two_point_percentage'] * 75 * pg['minutes_per_game'] / 36
    
    pg['three_pointers'] = pg['three_point_attempts'] * pg['three_point_percentage'] * 75 * pg['minutes_per_game'] / 36
    
    pg['free_throws'] = pg['free_throw_attempts'] * pg['free_throw_percentage'] * 75 * pg['minutes_per_game'] / 36

    pg['defensive_rebounds'] = pg['defensive_rebounds'] * 75 * pg['minutes_per_game'] / 36
    
    pg['offensive_rebounds'] = pg['offensive_rebounds'] * 75 * pg['minutes_per_game'] / 36
    
    pg['total_rebounds'] = pg['defensive_rebounds'] + pg['offensive_rebounds'] 
    
    pg['assists'] = pg['assists'] * 75 * pg['minutes_per_game'] / 36
    
    pg['steals'] = pg['steals'] * 75 * pg['minutes_per_game'] / 36
    
    pg['blocks'] = pg['blocks'] * 75 * pg['minutes_per_game'] / 36
    
    pg['turnovers'] = pg['turnovers'] * 75 * pg['minutes_per_game'] / 36
    
    pg['personal_fouls'] = pg['personal_fouls'] * 75 * pg['minutes_per_game'] / 36
    
    pg['points'] = 2 * pg['two_pointers'] + 3 * pg['three_pointers'] + 1 * pg['free_throws']
    
    pg['field_goals'] = pg['two_pointers'] + pg['three_pointers']
    
    pg['field_goal_percentage'] = pg['field_goals'] / ((pg['two_point_attempts'] + pg['three_point_attempts']) * 75 * pg['minutes_per_game'] / 36)
    
    final_columns = ['Player', 'Year', 'Age', 'minutes_per_game', 'field_goals', 'field_goal_percentage',
                     'three_pointers', 'three_point_percentage', 'two_pointers', 'two_point_percentage',  'free_throws',
                     'free_throw_percentage', 'total_rebounds', 'offensive_rebounds', 'defensive_rebounds',
                     'assists', 'steals', 'blocks', 'turnovers', 'personal_fouls', 'points']
    
    pg = pg.round(2)
    
    return pg[final_columns]

In [130]:
per_game_preds = {}

for player_id in career_preds.keys():

    per_game_preds[player_id] = per_game_averages(career_preds[player_id])

In [131]:
# Combine all the dataframes

switch = 0

for player_id in per_game_preds.keys():
    
    if not switch:
        per_game_df = per_game_preds[player_id]
        switch = 1
    
    else:
        per_game_df = pd.concat([per_game_df, per_game_preds[player_id]], axis = 0)

In [132]:
def stat_leaders(year, stat, N):
    
    stat_df = per_game_df[per_game_df['Year'] == year][['Player', stat]]
    return stat_df.sort_values(stat, ascending = False).head(N)

In [133]:
per_game_df[per_game_df['Year'] == 2022][['Player','Age','points']]

,Player,Age,points
index_col,,,
Aaron Gordon2022,Aaron Gordon,26.0,17.35
Austin Rivers2022,Austin Rivers,29.0,10.89
Bol Bol2022,Bol Bol,22.0,10.56
Facundo Campazzo2022,Facundo Campazzo,30.0,9.16
JaMychal Green2022,JaMychal Green,31.0,11.48
...,...,...,...
Patty Mills2022,Patty Mills,33.0,13.90
Quinndary Weatherspoon2022,Quinndary Weatherspoon,25.0,9.47
Rudy Gay2022,Rudy Gay,35.0,15.97


In [134]:
stat_leaders(2022, 'minutes_per_game', 10)

,Player,minutes_per_game
index_col,,
Julius Randle2022,Julius Randle,36.40
Russell Westbrook2022,Russell Westbrook,35.60
Bradley Beal2022,Bradley Beal,35.52
James Harden2022,James Harden,35.52
Domantas Sabonis2022,Domantas Sabonis,35.48
Jayson Tatum2022,Jayson Tatum,35.48
Pascal Siakam2022,Pascal Siakam,35.44
De'Aaron Fox2022,De'Aaron Fox,35.32
Damian Lillard2022,Damian Lillard,35.32


In [135]:
stat_leaders(2022, 'points', 10)

,Player,points
index_col,,
Bradley Beal2022,Bradley Beal,37.19
Joel Embiid2022,Joel Embiid,36.13
Luka Dončić2022,Luka Dončić,34.96
Damian Lillard2022,Damian Lillard,34.76
Stephen Curry2022,Stephen Curry,34.53
Devin Booker2022,Devin Booker,33.57
Jayson Tatum2022,Jayson Tatum,32.62
Zach LaVine2022,Zach LaVine,32.37
Donovan Mitchell2022,Donovan Mitchell,32.10


In [136]:
stat_leaders(2022, 'total_rebounds', 10)

,Player,total_rebounds
index_col,,
Clint Capela2022,Clint Capela,16.79
Rudy Gobert2022,Rudy Gobert,16.02
Jonas Valančiūnas2022,Jonas Valančiūnas,15.26
Joel Embiid2022,Joel Embiid,15.06
Giannis Antetokounmpo2022,Giannis Antetokounmpo,14.66
Nikola Jokić2022,Nikola Jokić,14.47
Domantas Sabonis2022,Domantas Sabonis,14.34
Andre Drummond2022,Andre Drummond,14.03
Jarrett Allen2022,Jarrett Allen,13.93


In [137]:
stat_leaders(2022, 'assists', 10)

,Player,assists
index_col,,
Russell Westbrook2022,Russell Westbrook,13.42
James Harden2022,James Harden,12.65
Luka Dončić2022,Luka Dončić,12.03
Trae Young2022,Trae Young,11.74
Chris Paul2022,Chris Paul,10.71
Draymond Green2022,Draymond Green,9.90
Damian Lillard2022,Damian Lillard,9.64
Nikola Jokić2022,Nikola Jokić,9.57
Ja Morant2022,Ja Morant,9.40


In [138]:
stat_leaders(2022, 'steals', 10)

,Player,steals
index_col,,
Jimmy Butler2022,Jimmy Butler,2.46
T.J. McConnell2022,T.J. McConnell,2.17
LaMelo Ball2022,LaMelo Ball,2.13
Ben Simmons2022,Ben Simmons,2.07
Matisse Thybulle2022,Matisse Thybulle,2.04
Kawhi Leonard2022,Kawhi Leonard,2.02
Draymond Green2022,Draymond Green,2.00
De'Aaron Fox2022,De'Aaron Fox,1.99
Larry Nance Jr.2022,Larry Nance Jr.,1.98


In [139]:
stat_leaders(2022, 'blocks', 10)

,Player,blocks
index_col,,
Myles Turner2022,Myles Turner,3.40
Rudy Gobert2022,Rudy Gobert,3.01
Clint Capela2022,Clint Capela,2.56
Jakob Poeltl2022,Jakob Poeltl,2.19
Chris Boucher2022,Chris Boucher,2.15
Nerlens Noel2022,Nerlens Noel,2.14
Robert Williams2022,Robert Williams,2.08
Jarrett Allen2022,Jarrett Allen,1.93
Mo Bamba2022,Mo Bamba,1.84


In [140]:
stat_leaders(2022, 'three_pointers', 10)

,Player,three_pointers
index_col,,
Stephen Curry2022,Stephen Curry,4.77
Damian Lillard2022,Damian Lillard,4.74
Buddy Hield2022,Buddy Hield,4.14
CJ McCollum2022,CJ McCollum,3.86
Fred VanVleet2022,Fred VanVleet,3.84
Zach LaVine2022,Zach LaVine,3.78
Donovan Mitchell2022,Donovan Mitchell,3.76
Duncan Robinson2022,Duncan Robinson,3.73
Terry Rozier2022,Terry Rozier,3.67


In [141]:
stat_leaders(2022, 'free_throws', 10)

,Player,free_throws
index_col,,
Joel Embiid2022,Joel Embiid,9.95
Trae Young2022,Trae Young,8.98
Jimmy Butler2022,Jimmy Butler,8.28
Bradley Beal2022,Bradley Beal,8.03
Damian Lillard2022,Damian Lillard,7.93
DeMar DeRozan2022,DeMar DeRozan,7.60
James Harden2022,James Harden,7.37
Giannis Antetokounmpo2022,Giannis Antetokounmpo,7.32
Kevin Durant2022,Kevin Durant,7.11


In [142]:
stat_leaders(2022, 'two_pointers', 10)

,Player,two_pointers
index_col,,
Joel Embiid2022,Joel Embiid,10.58
Bradley Beal2022,Bradley Beal,10.34
Nikola Jokić2022,Nikola Jokić,10.05
Anthony Davis2022,Anthony Davis,9.67
Giannis Antetokounmpo2022,Giannis Antetokounmpo,9.66
Devin Booker2022,Devin Booker,9.48
Zion Williamson2022,Zion Williamson,9.45
Bam Adebayo2022,Bam Adebayo,9.36
Collin Sexton2022,Collin Sexton,9.31


In [143]:
stat_leaders(2022, 'three_point_percentage', 10)

,Player,three_point_percentage
index_col,,
Seth Curry2022,Seth Curry,0.40
Cameron Payne2022,Cameron Payne,0.40
Paul George2022,Paul George,0.39
Gorgui Dieng2022,Gorgui Dieng,0.39
Danilo Gallinari2022,Danilo Gallinari,0.39
Norman Powell2022,Norman Powell,0.39
Kevin Durant2022,Kevin Durant,0.39
Kyrie Irving2022,Kyrie Irving,0.39
Jeremy Lamb2022,Jeremy Lamb,0.39


In [144]:
per_game_df[np.array(per_game_df['Player'] == 'Stephen Curry') & np.array(per_game_df['Year'] == 2022)]

,Player,Year,Age,minutes_per_game,field_goals,field_goal_percentage,three_pointers,three_point_percentage,two_pointers,two_point_percentage,free_throws,free_throw_percentage,total_rebounds,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points
index_col,,,,,,,,,,,,,,,,,,,,,
Stephen Curry2022,Stephen Curry,2022,33.0,34.4,11.63,0.47,4.77,0.39,6.86,0.53,6.49,0.89,7.17,0.86,6.31,7.45,1.65,0.29,4.08,2.72,34.53


In [145]:
per_game_df[per_game_df['Player'] == 'LeBron James']

,Player,Year,Age,minutes_per_game,field_goals,field_goal_percentage,three_pointers,three_point_percentage,two_pointers,two_point_percentage,free_throws,free_throw_percentage,total_rebounds,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points
index_col,,,,,,,,,,,,,,,,,,,,,
LeBron James2021,LeBron James,2021,36.0,33.44,12.68,0.51,3.13,0.36,9.55,0.59,5.30,0.70,10.38,0.84,9.54,10.52,1.46,0.77,5.02,2.16,33.79
LeBron James2022,LeBron James,2022,37.0,32.00,10.64,0.48,2.47,0.33,8.17,0.56,4.60,0.75,9.67,0.93,8.73,9.07,1.53,0.67,4.13,2.40,28.36
LeBron James2023,LeBron James,2023,38.0,31.56,9.44,0.46,2.46,0.33,6.99,0.53,4.18,0.76,9.01,1.05,7.96,8.15,1.51,0.66,3.75,2.63,25.52
LeBron James2024,LeBron James,2024,39.0,31.32,8.53,0.44,2.44,0.34,6.09,0.50,3.90,0.78,8.61,1.11,7.50,12.98,1.50,0.65,3.52,3.20,23.41
LeBron James2025,LeBron James,2025,40.0,31.16,8.19,0.43,2.43,0.34,5.76,0.49,3.65,0.78,8.44,1.10,7.34,11.75,1.49,0.58,3.38,3.38,22.45
LeBron James2026,LeBron James,2026,41.0,31.20,8.61,0.43,2.43,0.34,6.18,0.48,3.35,0.78,8.39,1.11,7.28,10.92,1.50,0.52,3.19,3.44,23.02


In [146]:
per_game_df[per_game_df['Player'] == 'Giannis Antetokounmpo']

,Player,Year,Age,minutes_per_game,field_goals,field_goal_percentage,three_pointers,three_point_percentage,two_pointers,two_point_percentage,free_throws,free_throw_percentage,total_rebounds,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points
index_col,,,,,,,,,,,,,,,,,,,,,
Giannis Antetokounmpo2021,Giannis Antetokounmpo,2021,26.0,33.00,13.37,0.57,1.44,0.30,11.94,0.64,8.52,0.68,14.44,2.13,12.31,7.63,1.58,1.58,4.40,3.58,36.71
Giannis Antetokounmpo2022,Giannis Antetokounmpo,2022,27.0,33.04,11.17,0.52,1.50,0.32,9.66,0.58,7.32,0.72,14.66,2.00,12.67,7.50,1.58,1.38,3.99,3.79,31.16
Giannis Antetokounmpo2023,Giannis Antetokounmpo,2023,28.0,32.88,10.33,0.51,1.54,0.33,8.79,0.56,7.18,0.74,14.45,1.85,12.60,7.26,1.58,1.30,3.84,3.63,29.37
Giannis Antetokounmpo2024,Giannis Antetokounmpo,2024,29.0,32.76,9.45,0.48,1.59,0.33,7.86,0.53,6.57,0.75,14.40,1.84,12.56,6.82,1.57,1.23,3.75,3.62,27.05
Giannis Antetokounmpo2025,Giannis Antetokounmpo,2025,30.0,32.52,8.57,0.47,1.59,0.34,6.98,0.51,6.05,0.77,14.30,1.83,12.47,6.71,1.56,1.22,3.59,3.59,24.78
Giannis Antetokounmpo2026,Giannis Antetokounmpo,2026,31.0,31.64,7.68,0.46,1.56,0.34,6.12,0.50,5.46,0.77,13.91,1.78,12.13,6.46,1.52,1.19,3.30,3.49,22.38


In [147]:
per_game_df[per_game_df['Player'] == 'Luka Dončić']

,Player,Year,Age,minutes_per_game,field_goals,field_goal_percentage,three_pointers,three_point_percentage,two_pointers,two_point_percentage,free_throws,free_throw_percentage,total_rebounds,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points
index_col,,,,,,,,,,,,,,,,,,,,,
Luka Dončić2021,Luka Dončić,2021,21.0,34.28,13.41,0.48,3.97,0.35,9.43,0.57,7.14,0.73,10.93,1.14,9.78,11.78,1.36,0.79,5.78,3.14,37.94
Luka Dončić2022,Luka Dončić,2022,22.0,35.00,12.23,0.46,3.49,0.34,8.73,0.54,7.01,0.76,10.57,1.24,9.33,12.03,1.53,0.73,4.67,3.35,34.96
Luka Dončić2023,Luka Dončić,2023,23.0,35.28,11.08,0.45,3.28,0.34,7.80,0.52,6.74,0.78,10.29,1.32,8.97,12.05,1.54,0.74,4.41,3.38,32.18
Luka Dončić2024,Luka Dončić,2024,24.0,35.32,10.46,0.44,3.20,0.34,7.25,0.50,6.28,0.78,9.79,1.32,8.46,12.14,1.62,0.74,4.19,3.38,30.40
Luka Dončić2025,Luka Dončić,2025,25.0,35.32,9.91,0.44,3.00,0.34,6.91,0.50,5.83,0.78,9.57,1.32,8.24,12.14,1.69,0.74,4.05,3.38,28.66
Luka Dončić2026,Luka Dončić,2026,26.0,35.20,9.47,0.43,2.79,0.34,6.67,0.49,5.69,0.78,9.53,1.32,8.21,12.10,1.69,0.73,3.89,3.37,27.42


In [148]:
per_game_df[per_game_df['Player'] == 'Chris Paul']

,Player,Year,Age,minutes_per_game,field_goals,field_goal_percentage,three_pointers,three_point_percentage,two_pointers,two_point_percentage,free_throws,free_throw_percentage,total_rebounds,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points
index_col,,,,,,,,,,,,,,,,,,,,,
Chris Paul2021,Chris Paul,2021,35.0,31.40,8.56,0.50,1.99,0.39,6.57,0.54,3.36,0.93,6.08,0.52,5.56,12.23,1.90,0.33,3.07,3.21,22.47
Chris Paul2022,Chris Paul,2022,36.0,30.24,7.65,0.47,2.03,0.38,5.62,0.52,2.83,0.88,5.48,0.69,4.79,10.71,1.76,0.32,2.90,3.28,20.16
Chris Paul2023,Chris Paul,2023,37.0,29.32,6.95,0.45,1.91,0.36,5.04,0.50,2.68,0.86,5.25,0.67,4.58,10.20,1.65,0.31,2.81,3.24,18.49
Chris Paul2024,Chris Paul,2024,38.0,27.84,6.42,0.44,1.79,0.36,4.63,0.49,2.43,0.84,4.81,0.64,4.18,9.74,1.57,0.29,2.67,3.07,17.07
Chris Paul2025,Chris Paul,2025,39.0,27.56,7.37,0.45,1.77,0.36,5.60,0.48,2.32,0.82,4.65,0.63,4.02,9.53,1.55,0.29,2.64,3.04,18.82
Chris Paul2026,Chris Paul,2026,40.0,27.64,6.94,0.44,1.77,0.36,5.17,0.48,2.40,0.82,4.55,0.63,3.92,9.44,1.55,0.29,2.65,3.05,18.06
